In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.seasonal import seasonal_decompose
import numpy as np



from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.utils import check_X_y, check_array
from sklearn.exceptions import NotFittedError
from scipy.sparse.linalg import lsmr
from sklearn.linear_model import LinearRegression,LogisticRegression

from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from sklearn.neural_network import MLPRegressor
from xgboost import XGBRegressor
from mlxtend.regressor import StackingRegressor,StackingCVRegressor
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import mean_squared_error

import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import pymannkendall as mk
import seaborn as sns

from statsmodels.graphics.tsaplots import plot_acf,plot_pacf



from statsmodels.tsa.arima.model import ARIMA


# Time Series Analysis Functions

## Reading Files

In [2]:
def Read_GNSS_Data(gnss_file_path):
    data = pd.read_csv(gnss_file_path)
    data['date'] = pd.to_datetime(data[['YEAR', 'MONTH', 'DAY']])
    data.set_index('date', inplace=True)
    return data

In [3]:
def Read_ERA5_Data(ERA5_file_path):
    ERA_5 = pd.read_csv(ERA5_file_path)
    ERA_5['date'] = pd.to_datetime(ERA_5['date'])
    ERA_5.set_index('date', inplace=True)
    return ERA_5

## Station Extraction

In [4]:

def Station_Extraction(STATION_NAME,data):
    return data[data['STN']== STATION_NAME]


In [5]:
def ERA5_STATION_EXTRACTION(STATION_NAME,ERA_5):
    return ERA_5[ERA_5['station']== STATION_NAME]
    

In [6]:
def Extract_individual_Year_of_Station(EXTRACTED_STATION,YEAR):
    return EXTRACTED_STATION[EXTRACTED_STATION['YEAR'] == YEAR]


## Correlation Analysis

In [7]:
def Corelation_Analysis(Station,data,ERA_5):
    ERA_STATION = ERA5_STATION_EXTRACTION(Station,ERA_5)
    GNSS_STATION = Station_Extraction(Station,data)
    plt.figure(figsize=(8, 2))
    plt.plot(ERA_STATION['ztd [m]'])
    plt.plot(GNSS_STATION['ZTD_igs'])
    plt.show()
    return ERA_STATION['ztd [m]'].corr(GNSS_STATION['ZTD_igs'])

## Stationarity Analysis

In [8]:
def Stationarity(STATION_NAME):
    result = adfuller(STATION_NAME['ZTD_igs'])
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

## Mankendall Trend Test Analysis

In [9]:
def Mankendall_Trend_Test(STATION_NAME):
    result = mk.hamed_rao_modification_test(STATION_NAME['ZTD_igs'])
    print('Trend:', result.trend)
    print('H:', result.h)
    print('p-value:', result.p)
    print('Z:', result.z)

## Plotting

In [10]:
def SinglePlot(EXTRACTED_STATION_NAME):
    
    fig, axs = plt.subplots(5, 2,figsize=(15, 14))
    annual_decomposition = sm.tsa.seasonal_decompose(EXTRACTED_STATION_NAME['ZTD_igs'], model='additive',period=30)
    semi_annual_decomposition = sm.tsa.seasonal_decompose(EXTRACTED_STATION_NAME['ZTD_igs'], model='additive',period=183)
    inter_annual_decomposition = sm.tsa.seasonal_decompose(EXTRACTED_STATION_NAME['ZTD_igs'], model='additive',period=365)
    
    df = EXTRACTED_STATION_NAME
    df['Inter-Annual'] = df['ZTD_igs'].diff(12)  # Assuming monthly data with a frequency of 12
    axs[0, 0].plot(EXTRACTED_STATION_NAME['ZTD_igs'].index, EXTRACTED_STATION_NAME['ZTD_igs'],color='red',label = 'ztd')
    axs[0, 0].set_title('ztd')

    axs[0, 1].plot(df.index, df['Inter-Annual'],color='orange',label = 'ztd')
    axs[0, 1].set_title('Inter Annual Variability')
    
    axs[1, 1].plot(annual_decomposition.trend.index, annual_decomposition.trend,color='blue',label = 'ztd')
    axs[1, 1].set_title('Annual Trend')
    axs[1, 0].plot(annual_decomposition.seasonal.index, annual_decomposition.seasonal,color='orange',label = 'ztd')
    axs[1, 0].set_title('Annual Seasonal')
    axs[2, 1].plot(semi_annual_decomposition.trend.index, semi_annual_decomposition.trend,color='blue',label = 'ztd')
    axs[2, 1].set_title('Semi Annual Trend')
    axs[2, 0].plot(semi_annual_decomposition.seasonal.index, semi_annual_decomposition.seasonal,color='orange',label = 'ztd')
    axs[2, 0].set_title('Semi Annual Seasonal')
   
    
    axs[3, 1].plot(inter_annual_decomposition.trend.index, inter_annual_decomposition.trend,color='blue',label = 'ztd')
    axs[3, 1].set_title('Inter Annual Trend')
    axs[3, 0].plot(inter_annual_decomposition.seasonal.index, inter_annual_decomposition.seasonal,color='orange',label = 'ztd')
    axs[3, 0].set_title('Inter Annual Seasonal')
    
    
    plot_pacf(EXTRACTED_STATION_NAME['ZTD_igs'],ax=axs[4, 0],color='red')
    axs[4, 0].set_title('Partial AutoCorrelation Analysis')
    axs[4, 0].set_ylabel('Partial Autocorrelation')
    axs[4, 0].set_xlabel('Lags')
    
    plot_acf(EXTRACTED_STATION_NAME['ZTD_igs'],ax=axs[4, 1],color='red')
    axs[4, 1].set_title('AutoCorrelation Analysis')
    axs[4, 1].set_ylabel('Autocorrelation')
    axs[4, 1].set_xlabel('Lags')
  
    fig.subplots_adjust(hspace=0.7)
    

In [11]:
def Subplotting_Individual_Years(Year_STATIONS,station):

    fig, axs = plt.subplots(2, 4,figsize=(15, 8), sharey=True)
# Plot data on subplots
    axs[0, 0].plot(Year_STATIONS[0].index, Year_STATIONS[0]['ZTD_igs'],color = 'red')
    axs[0, 0].set_title('2015')
    
    plt.setp(axs[0, 0].get_xticklabels(), rotation=45)

    axs[0, 1].plot(Year_STATIONS[1].index, Year_STATIONS[1]['ZTD_igs'],color = 'red')
    axs[0, 1].set_title('2016')
    plt.setp(axs[0, 1].get_xticklabels(), rotation=45)

    axs[1, 0].plot(Year_STATIONS[2].index, Year_STATIONS[2]['ZTD_igs'],color = 'red')
    axs[1, 0].set_title('2017')
    plt.setp(axs[1, 0].get_xticklabels(), rotation=45)

    axs[1, 2].plot(Year_STATIONS[3].index, Year_STATIONS[3]['ZTD_igs'],color = 'red')
    axs[1, 2].set_title('2018')
    plt.setp(axs[1, 2].get_xticklabels(), rotation=45)

    axs[1, 3].plot(Year_STATIONS[4].index, Year_STATIONS[4]['ZTD_igs'],color = 'red')
    axs[1, 3].set_title('2019')
    plt.setp(axs[1, 3].get_xticklabels(), rotation=45)

    axs[1, 1].plot(Year_STATIONS[5].index, Year_STATIONS[5]['ZTD_igs'],color = 'red')
    axs[1, 1].set_title('2020')
    plt.setp(axs[1, 1].get_xticklabels(), rotation=45)

    axs[0, 2].plot(Year_STATIONS[6].index, Year_STATIONS[6]['ZTD_igs'],color = 'red')
    axs[0, 2].set_title('2021')
    plt.setp(axs[0, 2].get_xticklabels(), rotation=45)

    axs[0, 3].plot(Year_STATIONS[7].index, Year_STATIONS[7]['ZTD_igs'],color = 'red')
    axs[0, 3].set_title('2022')
    plt.setp(axs[0, 3].get_xticklabels(), rotation=45)
    fig.suptitle(f'STATION {station}')
    fig.subplots_adjust(hspace=0.5)
    plt.show()

In [12]:
def Superimpose_Different_Years_Single_Plot(Year_STATIONS):
    plt.figure(figsize=(13, 6))
    plt.plot(Year_STATIONS[0].index,Year_STATIONS[0]['ZTD_igs'],color='red' )
    plt.plot(Year_STATIONS[1].index,Year_STATIONS[1]['ZTD_igs'],color='blue' )
    plt.plot(Year_STATIONS[2].index,Year_STATIONS[2]['ZTD_igs'],color='yellow' )
    plt.plot(Year_STATIONS[3].index,Year_STATIONS[3]['ZTD_igs'],color='purple' )
    plt.plot(Year_STATIONS[4].index,Year_STATIONS[4]['ZTD_igs'],color='pink' )
    plt.plot(Year_STATIONS[5].index,Year_STATIONS[5]['ZTD_igs'],color='green' )
    plt.plot(Year_STATIONS[6].index,Year_STATIONS[6]['ZTD_igs'],color='black' )
    plt.plot(Year_STATIONS[7].index,Year_STATIONS[7]['ZTD_igs'],color='orange' )

In [13]:
def Seasonal_TrendpPlotting(STATION_NAME):
    df_YEARLY = STATION_NAME.resample('m').sum()
    decomposition = sm.tsa.seasonal_decompose(df_YEARLY['ZTD_igs'], model='additive')
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8))
    decomposition.trend.plot(ax=ax1)
    ax1.set_title('Trend')
    decomposition.seasonal.plot(ax=ax2)
    ax2.set_title('Seasonal')
    
    plt.tight_layout()
    plt.show()
    

In [14]:
def Atmostpheric_Plot_With_GNSS(STATION,data,ERA_5):
    GNSS_STATION = Station_Extraction(STATION,data)
    ERAS_STATTION = ERA5_STATION_EXTRACTION(STATION,ERA_5)
    fig, axs = plt.subplots(2, 4,figsize=(20, 6))
    

    axs[0, 0].plot(GNSS_STATION['ZTD_igs'],color='red')
    axs[0, 0].set_title('ZTD_igs')
    
    plt.setp(axs[0, 0].get_xticklabels(), rotation=45)
#     plt.setp(ax2.get_xticklabels(), rotation=45)
    

    axs[0, 1].plot(ERAS_STATTION['T [K]'].index,ERAS_STATTION['T [K]'],color='blue')
    axs[0, 1].set_title('Atmospheric Temperature')
    plt.setp(axs[0, 1].get_xticklabels(), rotation=45)
   
    
    axs[0, 2].plot(ERAS_STATTION['p [hPa]'].index,ERAS_STATTION['p [hPa]'],color='orange')
    axs[0, 2].set_title('Atmospheric Pressure')
    plt.setp(axs[0, 2].get_xticklabels(), rotation=45)
   
    
    axs[0, 3].plot(ERAS_STATTION['p [hPa]'].index,ERAS_STATTION['e [hPa]'],color='black')
    axs[0, 3].set_title('Atmospheric Partial Water Vapour Pressure')
    plt.setp(axs[0, 3].get_xticklabels(), rotation=45)
   

In [15]:
def Mankendall_Trend_Test_Plot(STATION_NAME,data):
    p_values = []
    station_names = data ['STN'].drop_duplicates()
    plt.figure(figsize=(13, 6))
    for item in STATION_NAME:
        result = mk.hamed_rao_modification_test(item['ZTD_igs'])
        p_values.append (result.p)
    plt.bar(station_names.values, p_values,width=0.3)
    plt.xticks(rotation=45)  # Rotate the x-labels by 45 degrees
   
#     y_value = 10
#     plt.axhline(y=y_value, color='red', linestyle='--')
    plt.xlabel('Station Names')
    plt.ylabel('Mankendall P-values')
    plt.title('Trend Analysis with Mankendall Statistics Test')
    plt.show()


In [16]:
def Mankendall_Trend_Test_Cluster(STATION_NAME,data):
    cluster_values = []
    station_names = data ['STN'].drop_duplicates()
    plt.figure(figsize=(13, 6))
    for item in STATION_NAME:
        result = mk.hamed_rao_modification_test(item['ZTD_igs'])
        trend =result.trend
        if trend=='increasing':
            cluster_values.append(1)
        elif trend=='decreasing':
            cluster_values.append(-1)
        elif trend=='no trend':
            cluster_values.append(0)
    legend_mapping = { -1: "Decreasing Trend", 0: "No Trend", 1: "Increasing Trend"}
    renamed_labels = [legend_mapping[label] for label in cluster_values]
    dataframe = pd.DataFrame({'cluster_values':cluster_values,'station_names':station_names})
    sns.scatterplot(data=dataframe, x='station_names', y='cluster_values',hue=renamed_labels, palette='tab10')
    plt.xticks(rotation=45) 
#     plt.bar(station_names.values, p_values,width=0.3)
#     plt.xticks(rotation=45)  # Rotate the x-labels by 45 degrees
   
# #     y_value = 10
# #     plt.axhline(y=y_value, color='red', linestyle='--')
    plt.xlabel('Station Names')
    plt.ylabel('cluster Values')

In [17]:
def Trend_and_Stationarity_Visualisation(Stations,data):
    Mankendall_p_values = []
    stationarity_p_values = []
    fig, ax = plt.subplots(figsize=(13, 6))
    station_names = data ['STN'].drop_duplicates()
    for item in Stations:
        
        Mankendall_result = mk.hamed_rao_modification_test(item['ZTD_igs'])
        Mankendall_p_values.append ( Mankendall_result.p)
        
        Stationarity_result = adfuller(item['ZTD_igs'])
        stationarity_p_values.append(Stationarity_result[1])
        

    x = np.arange(len(station_names))
    
        
    ax.bar(x-0.3, Mankendall_p_values,width=0.3,color = 'red',label = 'Mankendal')
    ax.bar(x,stationarity_p_values,width=0.3,color ='blue',label = 'Stationarity')
    

    group_x = x - 0.3 / 2 
    ax.set_xticks(group_x)
    ax.set_xticklabels(station_names,rotation=45)

  
    
    
    ax.set_ylabel('P values')


    ax.set_title('Stationarity and Trend Visualisation')

 
    ax.legend()
    



In [18]:
def Corelation_Analysis_Plot(Station,data,ERA_5):
    correlation = []
    plt.figure(figsize=(15, 6))
    station_names = data ['STN'].drop_duplicates()
    for item in Station:
        ERA_STATION = ERA5_STATION_EXTRACTION(item,ERA_5)
        GNSS_STATION = Station_Extraction(item,data)
        correlation.append(ERA_STATION['ztd [m]'].corr(GNSS_STATION['ZTD_igs']))
    plt.bar(station_names, correlation,width=0.3)
    plt.xticks(rotation=45)  
    plt.xlabel('Station Names')
    plt.ylabel('Correlation Values')
    plt.title('Correlation Analysis with Adfuller Statistics Test')
    
    plt.show()

In [19]:
def GNSS_AND_ATMOSPHERIC_PARAMETER_CORRELATTION(Station,data,ERA_5):
    GNSS_Temperature_correlation = []
    GNSS_Pressure_correlation = []
    GNSS_Partial_Water_Vapour_Pressure_correlation = []
    fig, ax = plt.subplots(figsize=(13, 6))
    station_names = data ['STN'].drop_duplicates()
    for item in Station:
        ERA_STATION = ERA5_STATION_EXTRACTION(item,ERA_5)
        GNSS_STATION = Station_Extraction(item,data)
        GNSS_Temperature_correlation.append(ERA_STATION['T [K]'].corr(GNSS_STATION['ZTD_igs']))
        GNSS_Pressure_correlation.append(ERA_STATION['p [hPa]'].corr(GNSS_STATION['ZTD_igs']))
        GNSS_Partial_Water_Vapour_Pressure_correlation.append(ERA_STATION['e [hPa]'].corr(GNSS_STATION['ZTD_igs']))
    x = np.arange(len(station_names))
    
    ax.bar(x-0.25, GNSS_Temperature_correlation,width=0.25,color = 'red',label = 'GNSS AND TEMPERATURE CORELLATION')
    ax.bar(x,GNSS_Pressure_correlation,width=0.25,color ='blue',label = 'GNSSS AND PRESSURE CORRELATION')
    ax.bar(x+0.2,GNSS_Partial_Water_Vapour_Pressure_correlation,width=0.25,color ='green',label = 'GNSS AND PARTIAL WATER VAPOUR CORRELATION')
    

    group_x = x - 0.25 / 2 
    ax.set_xticks(group_x)
    ax.set_xticklabels(station_names,rotation=45)

  
    
    
    ax.set_ylabel('Correlation  values')


#     ax.set_title('CORRELATION BETWEEN GNSS(ZTD_igs), TEMPERATURE(T [K]), ATMOSPHERIC PRESSURE(p [hPa]) AND ATMOSPHERIC PARTIAL WATER VAPOUR PRESSURE(e [hPa])')

 
    ax.legend()

# Time Series Prediction

## Autogressive Forcasting

In [20]:
def Autoregressive_Forecasting(Station_Name,Lag):
    train_data = Station_Name.iloc[:-500]  # n is the number of data points to use for testing
    test_data = Station_Name.iloc[-500:]

    
    model = sm.tsa.AutoReg(train_data['ZTD_igs'], Lag)
  
    ar_model = model.fit()  # maxlag determines the lag order

    
    forecast = ar_model.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
    rmse = np.sqrt(mean_squared_error(test_data['ZTD_igs'].values.flatten(), forecast.values.flatten()))
    compare = pd.DataFrame({'Date':test_data['ZTD_igs'].index,'Actual':test_data['ZTD_igs'],'Predicted':forecast})
    plt.figure(figsize=(12, 8))
    plt.plot(test_data['ZTD_igs'],color ='yellow',label ='tested data')
    plt.plot(Station_Name['ZTD_igs'],color='blue',label = 'original data')
    plt.plot(forecast,color='red',label ='prediction')
#     ar_model.plot_predict(start=1, end=len(train_data['ZTD_igs'])-1)
    
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.legend()
    plt.xticks(rotation=45)
    return ar_model.summary(),compare.head(10),f'the root mean square is {rmse}',forecast

## Moving Average Forcasting

In [21]:
def Moving_Average(Station_Name, lag):
    train_data = Station_Name.iloc[:-500]  # n is the number of data points to use for testing
    test_data = Station_Name.iloc[-500:]
    model = ARIMA(train_data['ZTD_igs'], order=(0,0,lag))
    model_fit = model.fit()
    predictions = model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
    compare = pd.DataFrame({'Actual':test_data['ZTD_igs'],'Predicted':predictions })
    rmse = np.sqrt(mean_squared_error(test_data['ZTD_igs'].values.flatten(), predictions.values.flatten()))
   
    plt.figure(figsize=(12, 8))
    plt.plot(Station_Name['ZTD_igs'], label='Original Series')
    plt.plot(test_data['ZTD_igs'], label='tested Series',color='y')
    plt.plot(predictions, label='MA Predictions',color='r')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    return model_fit.summary(),compare.head(10),f'the root mean square is {rmse}',predictions

## ARIMA and SARIMA

In [22]:
def Arima_Forecasting(Station_Name,order):
    train_data = Station_Name.iloc[:-500]  # n is the number of data points to use for testing
    test_data = Station_Name.iloc[-500:]

    arima_model = ARIMA(train_data['ZTD_igs'], order=order)
    arima_model_fit = arima_model.fit()
    arima_model_fit.summary()
    arima_predictions = arima_model_fit.predict(start=len(train_data), end=len(train_data)+len(test_data)-1)
    rmse = np.sqrt(mean_squared_error(test_data['ZTD_igs'].values.flatten(), arima_predictions.values.flatten()))
    compare = pd.DataFrame({'Actual':test_data['ZTD_igs'],'Predicted':arima_predictions})
    plt.figure(figsize=(13,8))
    plt.plot(Station_Name['ZTD_igs'], label='Original Series')
    plt.plot(test_data['ZTD_igs'], label='tested data',color='y')
    plt.plot(arima_predictions, label='ARIMA Predictions',color='r')
    plt.legend()
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.xticks(rotation=45)
    return arima_model_fit.summary(),compare.head(10),f'the root mean square is {rmse}'

In [23]:
def check_Stationarity(differenced_data):
    result = adfuller(differenced_data)
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))

In [24]:
def check_Mankendall_Trend_Test(differenceddata):
    result = mk.hamed_rao_modification_test(differenceddata)
    print('Trend:', result.trend)
    print('H:', result.h)
    print('p-value:', result.p)
    print('Z:', result.z)

In [25]:
def Detrending(Station_name):
    fig, axs = plt.subplots(2, 2,figsize=(15, 14))
    axs[0,0].plot(Station_name['ZTD_igs'], label='Original Data')
    axs[0,0].set_xlabel('time')
    axs[0,0].set_ylabel('ZTD_igs')
    axs[0, 0].set_title('Original Data')
    
    
    trend_differenced_data = Station_name['ZTD_igs'].diff().dropna()
   
   
    
    axs[0,1].plot(trend_differenced_data, label='differenced')
    axs[0,1].set_xlabel('time')
    axs[0,1].set_ylabel('Z value')
    axs[0, 1].set_title('Differenced Data')
    
    plot_pacf(trend_differenced_data,ax=axs[1, 0],color='red')
    axs[1, 0].set_title('Partial AutoCorrelation Analysis')
    axs[1, 0].set_ylabel('Partial Autocorrelation')
    axs[1, 0].set_xlabel('Lags')
    
    plot_acf(trend_differenced_data,ax=axs[1, 1],color='red')
    axs[1, 1].set_title('AutoCorrelation Analysis')
    axs[1, 1].set_ylabel('Autocorrelation')
    axs[1, 1].set_xlabel('Lags')
    print(trend_differenced_data)
    return pd.DataFrame({'ZTD_igs':trend_differenced_data}),check_Mankendall_Trend_Test(trend_differenced_data),check_Stationarity(trend_differenced_data)

# pd.Series(differenced_data)




In [26]:
def Deseasonalise(Station_name):
    fig, axs = plt.subplots(2, 2,figsize=(15, 14))
    axs[0,0].plot(Station_name['ZTD_igs'], label='Original Data')
    axs[0,0].set_xlabel('time')
    axs[0,0].set_ylabel('Z value')
    axs[0, 0].set_title('Original Data')
    
    seasonal_differenced_data = Station_name['ZTD_igs'].diff(periods=366).dropna()
    
    
    axs[0,1].plot(seasonal_differenced_data, label='differenced')
    axs[0,1].set_xlabel('time')
    axs[0,1].set_ylabel('Z value')
    axs[0, 1].set_title('Differenced Data')
    
    plot_pacf(seasonal_differenced_data,ax=axs[1, 0],color='red')
    axs[1, 0].set_title('Partial AutoCorrelation Analysis')
    axs[1, 0].set_ylabel('Partial Autocorrelation')
    axs[1, 0].set_xlabel('Lags')
    
    plot_acf(seasonal_differenced_data,ax=axs[1, 1],color='red')
    axs[1, 1].set_title('AutoCorrelation Analysis')
    axs[1, 1].set_ylabel('Autocorrelation')
    axs[1, 1].set_xlabel('Lags')
    
    return pd.DataFrame({'ZTD_igs':seasonal_differenced_data}),check_Stationarity(seasonal_differenced_data)

In [27]:
def Remove_Trend_or_Seasonal(Station_Name,differencing_type):
    if differencing_type =='S':
        return Deseasonalise(Station_Name)
    elif differencing_type =='T':
        return Detrending(Station_Name)

# Machine_Learning_Forecasting

## Least Square Support vector Regression

In [28]:
class LSSVR(BaseEstimator, RegressorMixin):
    """Least Squares Support Vector Regression.

    Parameters
    ----------
    C : float, default=2.0
        Regularization parameter. The strength of the regularization is
        inversely proportional to C. Must be strictly positive.

    kernel : {'linear', 'rbf'}, default='linear'
        Specifies the kernel type to be used in the algorithm.
        It must be 'linear', 'rbf' or a callable.

    gamma : float, default = None
        Kernel coefficient for 'rbf'


    Attributes
    ----------
    support_: boolean np.array of shape (n_samples,), default = None
        Array for support vector selection.

    alpha_ : array-like
        Weight matrix

    bias_ : array-like
        Bias vector


    """

    def __init__(self, C=2.0, kernel='linear', gamma=None):
        self.C = C
        self.kernel = kernel
        self.gamma = gamma

    def fit(self, X, y, support=None):
        """Fit the model according to the given training data.
        Parameters
        ----------
        X : {array-like, sparse matrix} of shape (n_samples, n_features)
            Training data

        y : array-like of shape (n_samples,) or (n_samples, n_targets)
            Target values.

        support : boolean np.array of shape (n_samples,), default = None
            Array for support vector selection.

        Returns
        -------
        self : object
            An instance of the estimator.
        """

        X, y = check_X_y(X, y, multi_output=True, dtype='float')

        if not support:
            self.support_ = np.ones(X.shape[0], dtype=bool)
        else:
            self.support_ = check_array(support, ensure_2d=False, dtype='bool')

        self.support_vectors_ = X[self.support_, :]
        support_labels = y[self.support_]

        self.K_ = self.kernel_func(X, self.support_vectors_)
        omega = self.K_.copy()
        np.fill_diagonal(omega, omega.diagonal()+self.support_/self.C)

        D = np.empty(np.array(omega.shape) + 1)

        D[1:, 1:] = omega
        D[0, 0] = 0
        D[0, 1:] = 1
        D[1:, 0] = 1

        shape = np.array(support_labels.shape)
        shape[0] += 1
        t = np.empty(shape)

        t[0] = 0
        t[1:] = support_labels

        # TODO: maybe give access to  lsmr atol and btol ?
        try:
            z = lsmr(D.T, t)[0]
        except:
            z = np.linalg.pinv(D).T @ t

        self.bias_ = z[0]
        self.alpha_ = z[1:]
        self.alpha_ = self.alpha_[self.support_]

        return self

    def predict(self, X):
        """
        Predict using the estimator.
        Parameters
        ----------
        X : array-like or sparse matrix, shape (n_samples, n_features)
            Samples.

        Returns
        -------
        y : array-like of shape (n_samples,) or (n_samples, n_targets)
            Returns predicted values.
        """

        if not hasattr(self, 'support_vectors_'):
            raise NotFittedError

        X = check_array(X, ensure_2d=False)
        K = self.kernel_func(X, self.support_vectors_)
        return (K @ self.alpha_) + self.bias_

    def kernel_func(self, u, v):
        if self.kernel == 'linear':
            return np.dot(u, v.T)

        elif self.kernel == 'rbf':
            return rbf_kernel(u, v, gamma=self.gamma)

        elif callable(self.kernel):
            if hasattr(self.kernel, 'gamma'):
                return self.kernel(u, v, gamma=self.gamma)
            else:
                return self.kernel(u, v)
        else:
            # default to linear
            return np.dot(u, v.T)

    def score(self, X, y):
        from scipy.stats import pearsonr
        p, _ = pearsonr(y, self.predict(X))
        return p ** 2

    def norm_weights(self):
        A = self.alpha_.reshape(-1, 1) @ self.alpha_.reshape(-1, 1).T

        W = A @ self.K_[self.support_, :]
        return np.sqrt(np.sum(np.diag(W)))


# Stacked Ensemble Modelling

In [29]:
def Stack_ensemble_Prediction(Station_Name):
    plt.figure(figsize=(13, 6))

    
    X, y = Station_Name['ZWD'].to_numpy().reshape(-1, 1), Station_Name['ZTD_igs'].to_numpy().reshape(-1, 1)

    
  

    X_train, X_test, y_train, y_test = train_test_split(X[:-500], y[:-500], test_size=(len(y)-len(y[:-500])),random_state=0)

    lssvr_model = LSSVR()
    nn_model = MLPRegressor()
    xgb_model = XGBRegressor()
    
    stacked_model = StackingCVRegressor(
    regressors=[xgb_model, lssvr_model, nn_model],
    meta_regressor=LinearRegression()) 
    
    stacked_model.fit(X_train, y_train)

    y_pred = stacked_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    rmse = mse**0.5
    print(f"Root Mean Squared Error (RMSE): {rmse}")
    models = ['xgb_model','LS-SVR', 'Neural Network', 'Stacked Ensemble']
    color = ['red','blue','green','orange']
    score = []

    for model_score in [xgb_model,lssvr_model, nn_model, stacked_model]:
        cv_scores = -cross_val_score(model_score, X_train, y_train, scoring='neg_root_mean_squared_error')
        score.append(min(cv_scores))
        
    sns.barplot(models, score ,palette=color)
    plt.title('Base Model Performance')
    print(pd.DataFrame({'Date':Station_Name[-500:].index.values,'Actual':y_test.flatten(),'Predicted':y_pred}))
    return y_pred
 
    

In [30]:
# Stack_ensemble_Prediction(ADIS_STATION)


In [31]:
def Accuracy_Model_Comparisoms():
    fig, axs = plt.subplots(2, 2,figsize=(12, 8))
    Models = ['Stacked_model','Autoregressive','Moving Average','ARIMA']
    color = ['red','blue','green','orange']
    ABPO_accuracy_Score =[0.004362919300503391,0.04863937221224741,0.04921956963363566,0.048943436374477275]
    
    ADIS_accuracy_Score = [0.002155871885795152,0.02779981926741842,0.027985644339273124,0.029457628455895148]
    
    MAL2_accuracy_Score=[0.005190404870723981,0.039897567581271665,0.040397887293639065,0.03997494748605169]
    
    NURK_accuracy_Score = [ 0.0003102568194383033,0.04961848133128715,0.051115772656624314,0.05462753868196547]
    
    axs[0, 0].bar(Models, ABPO_accuracy_Score,color = color)
    axs[0, 0].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION ABPO')
    
    plt.setp(axs[0, 0].get_xticklabels(), rotation=45)
    
    axs[0, 1].bar(Models, ADIS_accuracy_Score,color = color)
    axs[0, 1].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION ADIS')
    plt.setp(axs[0, 1].get_xticklabels(), rotation=45)
    
    axs[1, 0].bar(Models, MAL2_accuracy_Score,color = color)
    axs[1, 0].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION MAL2')
    plt.setp(axs[1, 0].get_xticklabels(), rotation=45)
    
    axs[1, 1].bar(Models, NURK_accuracy_Score,color = color)
    axs[1, 1].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION NURK')
    plt.setp(axs[1, 1].get_xticklabels(), rotation=45)
    
    for index,value in enumerate(ABPO_accuracy_Score):
        axs[0, 0].text(index, value, str(round(value,4)), ha='center', va='bottom',fontsize=12) if index == 0 else axs[0, 0].text(index, value-0.01, str(round(value,4)), ha='center', va='bottom',fontsize=12)
    for index,value in enumerate(ADIS_accuracy_Score):
        axs[0, 1].text(index, value, str(round(value,4)), ha='center', va='bottom',fontsize=12) if index==0 else axs[0, 1].text(index, value-0.006, str(round(value,4)), ha='center', va='bottom',fontsize=12)
    for index,value in enumerate(MAL2_accuracy_Score):
        axs[1, 0].text(index, value, str(round(value,4)), ha='center', va='bottom',fontsize=12) if index ==0 else axs[1, 0].text(index, value-0.007, str(round(value,4)), ha='center', va='bottom',fontsize=12)
    for index,value in enumerate(NURK_accuracy_Score):
        axs[1, 1].text(index, value, str(round(value,4)), ha='center', va='bottom' ,fontsize=12) if index == 0 else axs[1, 1].text(index, value-0.01, str(round(value,4)), ha='center', va='bottom' ,fontsize=12)

    fig.subplots_adjust(hspace=0.7)
    fig.suptitle('MODEL ACCURACY ASSESSMENT BASED ON ROOT MEAN SQUARE ERROR')
    
    

In [32]:
def Correlate_Time_Series_with_Stacked_Ensemble(Station_Name):
    station1_predicted_values = pd.Series(Stack_ensemble_Prediction(Station_Name))
    station2_predicted_values = Autoregressive_Forecasting(Station_Name,1)[3]
   
    plt.figure(figsize=(8, 2))
    plt.plot(station1_predicted_values,station2_predicted_values)
#     plt.plot(station2_predicted_values)
    plt.show()
    print(station1_predicted_values,station2_predicted_values)
   
    return station1_predicted_values.corr(station2_predicted_values)
    

In [33]:
data = Read_GNSS_Data("IGS_ALL_DATA_DAILY_OSAH_imputed.csv")
ABPO_STATION = Station_Extraction('ABPO',data)
ADIS_STATION = Station_Extraction('ADIS',data)
MAL2_STATION = Station_Extraction('MAL2',data)
NURK_STATION = Station_Extraction('NURK',data)

In [34]:
# Correlate_Time_Series_with_Stacked_Ensemble(ADIS_STATION)

In [35]:
def Accuracy_Model_Comparisoms_AIC():
    fig, axs = plt.subplots(2, 2,figsize=(12, 8))
    Models = ['Autoregressive','Moving Average','ARIMA']
    color = ['blue','green','orange']
    ABPO_accuracy_Score =[-12409.075,-10126.322,-12705.859]
    
    ADIS_accuracy_Score = [-14752.085,-12359.645,-14882.397]
    
    MAL2_accuracy_Score=[-12390.739,-10856.631,-12539.575]
    
    NURK_accuracy_Score = [ -13318.719,-9837.557,-13347.311]
    
    axs[0, 0].bar(Models, ABPO_accuracy_Score,color = color)
    axs[0, 0].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION ABPO')
    
    plt.setp(axs[0, 0].get_xticklabels(), rotation=45)
    
    axs[0, 1].bar(Models, ADIS_accuracy_Score,color = color)
    axs[0, 1].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION ADIS')
    plt.setp(axs[0, 1].get_xticklabels(), rotation=45)
    
    axs[1, 0].bar(Models, MAL2_accuracy_Score,color = color)
    axs[1, 0].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION MAL2')
    plt.setp(axs[1, 0].get_xticklabels(), rotation=45)
    
    axs[1, 1].bar(Models, NURK_accuracy_Score,color = color)
    axs[1, 1].set_title('PREDICTION ACCURACY ASSESSMENT AT STATION NURK')
    plt.setp(axs[1, 1].get_xticklabels(), rotation=45)
    
    for index,value in enumerate(ABPO_accuracy_Score):
        axs[0, 0].text(index, value, str(round(value,4)), ha='center', va='bottom',fontsize=12) if index == 0 else axs[0, 0].text(index, value-0.01, str(round(value,4)), ha='center', va='bottom',fontsize=12)
    for index,value in enumerate(ADIS_accuracy_Score):
        axs[0, 1].text(index, value, str(round(value,4)), ha='center', va='bottom',fontsize=12) if index==0 else axs[0, 1].text(index, value-0.006, str(round(value,4)), ha='center', va='bottom',fontsize=12)
    for index,value in enumerate(MAL2_accuracy_Score):
        axs[1, 0].text(index, value, str(round(value,4)), ha='center', va='bottom',fontsize=12) if index ==0 else axs[1, 0].text(index, value-0.007, str(round(value,4)), ha='center', va='bottom',fontsize=12)
    for index,value in enumerate(NURK_accuracy_Score):
        axs[1, 1].text(index, value, str(round(value,4)), ha='center', va='bottom' ,fontsize=12) if index == 0 else axs[1, 1].text(index, value-0.01, str(round(value,4)), ha='center', va='bottom' ,fontsize=12)

    fig.subplots_adjust(hspace=0.7)
    fig.suptitle('MODEL ACCURACY ASSESSMENT BASED ON  AKAIKE INFORMATION CRITERIA(AIC)')
    
    